<a href="https://colab.research.google.com/github/amrahmani/NN/blob/main/Ch5_LSTM_WeatherPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem: A one-day-ahead temperature prediction problem**


Using PyTorch, create an **LSTM** and write code to train the model based on this data (https://raw.githubusercontent.com/amrahmani/Pythorch/main/WeatherHistory.csv), and test it for the problem. The model has four inputs: temperature (°C), humidity, wind speed (km/h), and pressure (millibars), respectively, and one output: "one-hour ahead temperature (°C)".

**Import Libraries:**

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

**Load and Prepare Dataset**

In [3]:
# Load dataset
url = 'https://raw.githubusercontent.com/amrahmani/Pythorch/main/WeatherHistory.csv'

# Read the CSV file into a Pandas DataFrame (assuming no header row)
data = pd.read_csv(url)

# Assuming your dataset columns are: 'Temperature', 'Humidity', 'Wind Speed', 'Pressure', 'Six-hour ahead Temperature'
# Separate inputs and outputs
X = data[["Temperature (C)", "Humidity", "Wind Speed (km/h)", "Pressure (millibars)"]].values
y = data[['one-hour ahead Temperature (C)']].values

# Normalization
# Scale the input features and target variable to the range [0, 1]; X_norm = (X - X_min) / (X_max - X_min)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Convert the numpy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Use the last 21 to 1 rows as test data
X_test = X[-21:-1]
y_test = y[-21:-1]

# Use the remaining data as training data
X_train = X[:-21]
y_train = y[:-21]

# Create a DataLoader for the training data to facilitate mini-batch training
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


**Define and Train the LSTM**

In [8]:
# Define the LSTM model class
class WeatherLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(WeatherLSTM, self).__init__()
        # LSTM layer: input size, hidden size
        # The difference between RNNs and LSTMs is that LSTMs don't have a specific "number_of_layers" to define their class
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        # Fully connected layer: hidden size to output size (1)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden and cell states with zeros
        h_0 = torch.zeros(1, x.size(0), hidden_size)
        c_0 = torch.zeros(1, x.size(0), hidden_size)
        # LSTM forward pass
        out, _ = self.lstm(x, (h_0, c_0)) # It returns two tensors, and assigns the first tensor to out and discards the second one (might contain the final hidden state) using the _
        # Fully connected layer
        out = self.fc(out[:, -1, :]) #  It selects all elements of the 1st dimension (:), the last element (-1) of the 2nd dimension, and all elements of the 3rd dimension (:)
        return out


# Hyperparameters
input_size = 4      # Number of input features
hidden_size = 96    # Number of features in the hidden state
output_size = 1     # Single output (next hour temperature)
num_epochs = 100    # Number of epochs to train
learning_rate = 0.001  # Learning rate

# Initialize the model, loss function, and optimizer
model = WeatherLSTM(input_size, hidden_size, output_size)

criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

# Training loop
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # Reshape input to (number of elements in the 1st dimention, sequence_length, input_size)
        X_batch = X_batch.view(X_batch.size(0), 1, input_size)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.0225
Epoch [20/100], Loss: 0.0073
Epoch [30/100], Loss: 0.0035
Epoch [40/100], Loss: 0.0009
Epoch [50/100], Loss: 0.0051
Epoch [60/100], Loss: 0.0017
Epoch [70/100], Loss: 0.0014
Epoch [80/100], Loss: 0.0049
Epoch [90/100], Loss: 0.0042
Epoch [100/100], Loss: 0.0014


**Test the LSTM and Compare Predictions**

In [7]:
# When we don't need to compute gradients during the forward pass
with torch.no_grad():
    X_test = X_test.view(X_test.size(0), 1, input_size)
    predictions = model(X_test)
    actuals = y_test

    # Rescale predictions and actuals back to original scale (Denormalization)
    predictions = scaler_y.inverse_transform(predictions)
    actuals = scaler_y.inverse_transform(actuals)

    # Compare predictions and actual values
    print('Predictions vs   Actuals         Comparison')
    for i in range(len(predictions)):
      print(predictions[i], '\t', actuals[i], '\t', predictions[i]-actuals[i])


Predictions vs   Actuals         Comparison
[10.07328649] 	 [9.46000015] 	 [0.61328633]
[9.58767553] 	 [10.68000004] 	 [-1.09232451]
[10.73967732] 	 [8.93999999] 	 [1.79967733]
[8.92116248] 	 [8.93999999] 	 [-0.01883751]
[9.04982965] 	 [12.49999986] 	 [-3.45017021]
[12.54033519] 	 [16.03000026] 	 [-3.48966508]
[16.06464402] 	 [19.86000005] 	 [-3.79535603]
[19.92818247] 	 [21.87999945] 	 [-1.95181699]
[21.93550117] 	 [22.80000035] 	 [-0.86449918]
[22.95498424] 	 [23.96000013] 	 [-1.00501589]
[23.97412428] 	 [24.92000032] 	 [-0.94587604]
[24.90652184] 	 [24.96999983] 	 [-0.06347799]
[24.9550453] 	 [25.03999976] 	 [-0.08495446]
[24.996814] 	 [24.82999996] 	 [0.16681403]
[24.67560588] 	 [22.22000046] 	 [2.45560542]
[22.13071562] 	 [18.86999999] 	 [3.26071563]
[18.85497639] 	 [17.13999976] 	 [1.71497663]
[17.14409111] 	 [14.9300002] 	 [2.2140909]
[15.00469448] 	 [15.41999974] 	 [-0.41530526]
[15.44390914] 	 [14.17999981] 	 [1.26390933]
